# Practical 4

In [1]:
#pip install cvxopt
import numpy as np
import cvxopt
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier

## Task 0: The Data

We will work with the data from practical 3. Load the data and split it into a training and test set. You can re-use the data splitting function from Practical 2.

In [145]:
def split_data(X, y, frac=0.3, seed=None):
    if seed is not None:
        np.random.seed(seed)

        
        data = np.copy(X)
        targets = np.copy(y)
        #shuffling the indexes becouse both the train and test data needs to be shuffled
        shuffled_idx = np.random.permutation(data.shape[0])

        X_train = data[shuffled_idx][0:int(data.shape[0]*0.7)]
        y_train = targets[shuffled_idx][0:int(data.shape[0]*0.7)]

        X_test = data[shuffled_idx][int(data.shape[0]*0.7):]
        y_test = targets[shuffled_idx][int(data.shape[0]*0.7):]
        return X_train,X_test,y_train,y_test


In [152]:
def preprocess(data,mean=None,std=None):
    if mean is None and std is None:

        mean = np.mean(data,axis=0)
        std = np.std(data,axis=0)
    
    else:
        mean = mean
        std = std
    processed_data = (data-mean)/std
        
    return processed_data

In [153]:
# load data
X_2d, t_2d = np.load('data/nonlin_2d_data.npy')[:,:2], np.load('data/nonlin_2d_data.npy')[:, 2]

In [155]:
# split data
X_train, X_test, t_train, t_test = split_data(X_2d, t_2d, seed=1)

train_mean = np.mean(X_train,axis=0)
train_std = np.std(X_train,axis=0)


norm_X_train = preprocess(X_train)

norm_X_test = preprocess(X_test,mean=train_mean,std=train_std)

In [156]:
print (X_train.shape)

(175, 2)


## Task 1: Support Vector Machines

First, you will implement a training algorithm for the Support Vector Machine (SVM). For solving the quadratic program, we provide a simple interface to the cvxopt library below.

In SVMs, each data sample $x_n$ has a corresponding lagrange multiplier $\alpha_n$ which indicates if $x_n$ is a support vector. In the latter case $\alpha_n > 0$ holds. 
The goal of learning the SVM is to figure out which samples are support vectors by learning $\mathbf{\alpha}$. The dual SVM optimizes the following quadratic program.

$$ \min \frac{1}{2} \sum_{n=1}^N \sum_{m=1}^N \alpha_n \alpha_m t_n t_m k(\mathbf{x}_n, \mathbf{x}_m) - \sum_{n=1}^N \alpha_n$$
subject to 
$$ 0 \leq \alpha_n \leq C $$
$$ \sum_{n=1}^N \alpha_n t_n = 0 $$ 

The quadratic program solver expects the following form:
$$ \min \frac{1}{2} \alpha^T P \alpha + \mathbf q^T \mathbf \alpha $$
subject to 

$$A \alpha = b$$
$$G \alpha \leq h $$

Here, $A$ and $G$ are matrices with one row per individual constraint. Similarly, $b$ and $h$ are vectors with one element per individual constraint.

Having trained the SVM, a prediction for an input $\mathbf{x}$ is made by:

$$ y = sign([\sum_n^{N} \alpha_n t_n k(\mathbf{x}, \mathbf{x}_n)] + b)  $$


### Task 1.1
 
Use the code provided below as a basis to express the constrained optimization problem in terms of $P, q, A, b, G$ and $h$ and implement a function `fit_svm` which passes these variables to the provided QP solver. Fit a SVM on the training data and extract its parameters.

**Hints:**
  - The box constraint $0 \leq \alpha_n \leq C$ defines two constraints of the form $G \alpha_n \leq h$ for each $\alpha_n$.
  - The inequality $x \geq 0$ is equivalent to $-x \leq 0$.

In [218]:
def fit_svm(X, t, kernel, C=1.0):
    '''Fit SVM using data (X,t), specified kernel and parameter C.'''

    t = np.array([-1 if l == 0 else 1 for l in t])
    P = np.zeros((len(X), len(X)))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            k = kernel(X[i],X[j])
            k_t = t[i]*t[j]*k
            P[i,j] = k_t
            
    q = -1*np.ones((X.shape[0],1))
    A = (t.T).reshape(-1,len(X)).astype(float)
    b = np.zeros((1, 1))  
    #print (b.shape[1])
    G = np.concatenate((np.eye(X.shape[0]), -1*np.eye(X.shape[0])),axis=0)
    h = np.concatenate((C*np.ones((X.shape[0])), np.zeros((X.shape[0]))),axis=0)
    #h = np.concatenate((np.zeros((X.shape[0])),C*np.ones((X.shape[0]))),axis=0)

    assert P.shape == (len(X), len(X))
    assert len(q) == len(X)
    assert A.shape == (1, len(X)) and A.dtype == 'float'
    assert b.shape == (1, 1)
    assert len(G) == 2 * len(X)
    assert len(h) == 2 * len(X)

    return solve_quadratic_program(P, q, A, b, G, h)

def solve_quadratic_program(P, q, A, b, G, h):
    '''Uses cvxopt to solve the quadratic program.'''
    P, q, A, b, G, h = [cvxopt.matrix(var) for var in [P, q, A, b, G, h]]
    minimization = cvxopt.solvers.qp(P, q, G, h, A, b)
    lagr_mult = np.ravel(minimization['x'])
    return lagr_mult


def extract_parameters(X, t, kernel, lagr_mult, threshold=1e-7):
    '''Computes the intercept from the support vector constraints.
    
    Inputs
        X:         predictors
        t:         targets
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        threshold: threshold for choosing support vectors
    
    Returns
        lagr_mult: lagrange multipliers for the support vectors
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept
    '''
    t = np.array([-1 if l == 0 else 1 for l in t])
    
    # ---------------- INSERT CODE ----------------------
    
    lagr_mult_ = lagr_mult[lagr_mult>0]
    svs = X[lagr_mult>0]
    sv_labels = t[lagr_mult>0]
    weights = ((lagr_mult * t).T@X)
    intercept = t[lagr_mult>0] - svs @weights
    # ---------------- END CODE -------------------------

    return lagr_mult_, svs, sv_labels, intercept


In [219]:
def rbf_kernel(x1,x2,sigma=4):
    return np.exp((-1 * np.linalg.norm(x1 - x2) ** 2) / (2 * sigma))

In [220]:
# Fit SVM on training data

lagr_mult = fit_svm(X_train,t_train,rbf_kernel)

# Extract parameters

lagr_mult, svs, sv_labels, intercept = extract_parameters(X_train, t_train, rbf_kernel, lagr_mult)



     pcost       dcost       gap    pres   dres
 0: -1.7199e+01 -3.0115e+02  1e+03  2e+00  1e-15
 1: -9.9615e+00 -1.5516e+02  2e+02  8e-02  7e-16
 2: -1.4238e+01 -3.7492e+01  2e+01  1e-02  8e-16
 3: -1.7975e+01 -2.6140e+01  8e+00  3e-03  7e-16
 4: -1.9043e+01 -2.3176e+01  4e+00  1e-03  5e-16
 5: -1.9495e+01 -2.2210e+01  3e+00  6e-04  6e-16
 6: -2.0056e+01 -2.1189e+01  1e+00  1e-04  6e-16
 7: -2.0361e+01 -2.0696e+01  3e-01  2e-05  6e-16
 8: -2.0479e+01 -2.0523e+01  4e-02  2e-06  6e-16
 9: -2.0495e+01 -2.0500e+01  5e-03  2e-08  8e-16
10: -2.0497e+01 -2.0497e+01  2e-04  5e-10  8e-16
11: -2.0497e+01 -2.0497e+01  3e-06  7e-12  7e-16
Optimal solution found.


### Task 1.2

Having learnt an SVM, we can use the calculated parameters to make predictions on novel samples.
- Implement a function `svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept)`.
- Use this function with your Gaussian RBF kernel (Practical 3) and compute the test accuracy on the 2d dataset.

In [221]:
def svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept):
    
    # ---------------- INSERT CODE ----------------------
    
    kernel_matrix = np.zeros((len(X), len(svs)))
    for i in range(X.shape[0]):
        for j in range(svs.shape[0]):
            k = kernel(X[i],svs[j])
            kernel_matrix[i,j] = k
    
    print(svs.shape)
    prediction = (sv_labels*lagr_mult) @ kernel_matrix.T  + intercept[0]



    # ---------------- END CODE -------------------------
    
    return np.sign(prediction)

In [223]:
# Calculate test accuracy

#print (intercept)
pred = svm_predict(X_test,rbf_kernel, lagr_mult, svs, sv_labels, intercept)
pred[pred<0]=0
print (np.sum(t_test==pred)/len(pred))

(175, 2)
0.6133333333333333


### Task 1.3

- Instead of using the Gaussian RBF kernel, use the linear kernel (dot product) defined in Practical 3.
- Compare results on with both kernels with sklearn implementation (SVC)
- Visualize the predictions on the test set, the learned support vectors and the decision boundary for both kernels.

In [224]:
def linear_kernel(a, b):
    return a.T@b

In [225]:
# Fit SVM with linear kernel and calculate the test accuracy
pred_linear = svm_predict(X_test,linear_kernel, lagr_mult, svs, sv_labels, intercept)
pred_linear[pred_linear<0]=0
print (np.sum(t_test==pred_linear)/len(pred_linear))

(175, 2)
0.56


In [13]:
# Fit SVM using sklearn and calculate the test accuracy


In [14]:
# Visualize


# Task 2: Decision Trees

Next, we will implement a simple decision tree classifier using the Wine dataset, one of the standard sklearn datasets. 

We will use the Gini impurity as a criterion for splitting. It is defined for a set of labels as
$$ G = \sum_{i=0}^C p(i) * (1- p(i)) $$

Given labels $l$ and split $l_a$ and $l_b$, the weighted removed impurity can be computed by $G(l) - \frac{|l_a|}{|l|}G(l_a) - \frac{|l_b|}{|l|}G(l_b)$.

Here is a simple explanation of the Gini impurity that you may find useful: https://victorzhou.com/blog/gini-impurity/


### Task 2.1

1. Plot the distribution of the first feature of for each class of the wine dataset.
2. Implement a function `gini_impurity(t)` that computes the Gini impurity for an array of labels `t`.
3. Calculate the removed Gini impurity for a split after 50 samples, i.e. between `t[:50]` and `t[50:]`.

In [226]:
# Load Wine dataset and split into train+test set

X, t = load_wine(return_X_y=True)
print(X.shape, t.shape)
X_train, X_test, t_train, t_test = split_data(X, t, seed=1)

(178, 13) (178,)


In [227]:
# Plot distribution


In [252]:
# Compute Gini impurity
def gini_imp(t):
    gini=0
    n_classes = len(np.unique(t))
    p = np.zeros(((n_classes),))
    for j in range(n_classes):
        p[j]= len(t[t==j])/(len(t))
        
    for c in range(n_classes):
        gini = gini + p[c]*(1-p[c])
    
    return gini



### Task 2.2
Split the data along the first 12 features and plot the removed Gini impurity at different values of this feature. Which is the optimal split?

In [253]:
# Plotting


### Task 2.3

1. Implement a function `build_tree(X, t, depth)` which recursively builds a tree. Use the classes `Node` and `Leaf` as a data structure to build your tree.
2. Implement a function `predict_tree(tree, x)` which makes a prediction for sample `x`. Obtain scores for the `wine` dataset and compare to `sklearn.tree.DecisionTree`.
3. Switch back to the synthetic 2d dataset from the beginning (kernel methods). Compute scores and visualize the decisions in a 2d grid.

In [254]:
class Node:
    def __init__(self, left, right, n_feat, threshold):
        self.left = left
        self.right = right
        self.n_feat = n_feat
        self.threshold = threshold


class Leaf:
    def __init__(self, label):
        self.label = label


In [255]:
def gini_best_split(X,t):
    gini_impurities = {}
    for feat in range(X.shape[1]):
        for val in X[:,feat]:
            split_r = t[X[:,feat]>val]
            split_l = t[X[:,feat]<=val]
            
            gini_r = gini_imp(split_r)
            gini_l = gini_imp(split_l)
            gini_total = gini_r + gini_l
            gini_impurities[feat,val] = gini_total
            
    best_split = min(gini_impurities,key=gini_impurities.get)
    return best_split

In [300]:
# Implement recursive tree function

def build_tree(X, t, depth, max_depth=3, n_labels=2):
    
    
    # ---------------- INSERT CODE ----------------------
    
    X_subset = X.copy()
    t_subset = t.copy()
    
    best_split_feat, best_split_val  = gini_best_split(X_subset,t_subset)
    #fixing values for the left and right nodes

    X_left  = X_subset[X_subset[:,best_split_feat]<=best_split_val].copy()
    t_left = t_subset[X_subset[:,best_split_feat]<=best_split_val].copy()
    X_right = X_subset[X_subset[:,best_split_feat]>best_split_val].copy()
    t_right = t_subset[X_subset[:,best_split_feat]>best_split_val].copy()
    
    if depth<max_depth and (len(X_left)>1 and len(X_right>1)):
        #making the nodes
        #level = Node(left_node, right_node, n_feat, best_split_val)
        depth = depth+1
        print (depth)
        left_node = build_tree(X_left, t_left, depth, max_depth, n_labels)
        right_node = build_tree(X_right, t_right, depth, max_depth, n_labels)
        node = Node(left_node, right_node, best_split_feat, best_split_val)
    else:
        # making the leafs
        left_label = np.argmax([np.count_nonzero(t_left==l) for l in range(n_labels)])
        left_leaf = Leaf(left_label)
        right_label = np.argmax([np.count_nonzero(t_right==l) for l in range(n_labels)])
        right_leaf = Leaf(right_label)
        node = Node(left_leaf, right_leaf, best_split_feat, best_split_val)
        
    return node

    # ---------------- END CODE -------------------------

    
def predict_tree(node, x):
    
    # ---------------- INSERT CODE ----------------------
    traversed_node = node
    while not type(traversed_node)==Leaf:
        
        feat_idx = traversed_node.n_feat
        feat_threshold = traversed_node.threshold
        if x[feat_idx]<=feat_threshold:
            traversed_node = traversed_node.left
        else:
            traversed_node = traversed_node.right
        
    return traversed_node.label
        
    # ---------------- END CODE -------------------------

In [302]:
# Build tree

tree = build_tree(X_train, t_train, 0, max_depth=3, n_labels=2)



1
2
3
2
3
3


In [303]:
# Calculate training and test scores

decision_train_pred = [predict_tree(tree,x) for x in X_train]

In [304]:
print (np.sum(decision_train_pred==t_train)/len(t_train))

0.75


In [308]:
decision_test_pred = [predict_tree(tree,x) for x in X_test]

In [309]:
print (np.sum(decision_test_pred==t_test)/len(t_test))

0.5925925925925926


In [ ]:
# Calculate test score using sklearn


In [ ]:
# Calculate test score for synthetic 2D dataset


In [ ]:
# Visualize
